In [7]:
!pip install metpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import glob
import pandas as pd
from zipfile import ZipFile
from datetime import datetime
from metpy.io.metar import *
import re
import metpy

In [4]:
metpy.__version__

'1.6.2'

In [5]:
if 'data-science-challenge-at-eef-2024.zip' not in glob.glob('*'):
    !kaggle competitions download -c data-science-challenge-at-eef-2024

    with ZipFile('data-science-challenge-at-eef-2024.zip') as zip:
        zip.extractall()

In [61]:
df = pd.read_csv('public.csv')
print(df.shape)
df.head()

(302399, 10)


,flightid,hora_ref,origem,destino,url_img_satelite,metaf,metar,prev_troca_cabeceira,troca_cabeceira_hora_anterior,espera
0,504a62621cd231d6ab67e674ce538cd3,2022-06-01T01:00:00Z,SBCF,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
1,b0fd0f83644625ecc21f5261e8e5e347,2022-06-01T01:00:00Z,SBPA,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
2,1210f0ca07ddca00d09a3e02d3b100d8,2022-06-01T01:00:00Z,SBSP,SBCF,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCF 010000Z 12006KT CAVOK 21/14 Q1018=,0,0,0.0
3,b25032f34507cce285ee779446496568,2022-06-01T01:00:00Z,SBGR,SBCT,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,0,0,0.0
4,00762a9892ecba7c66d1d87800d38cac,2022-06-01T01:00:00Z,SBSP,SBSV,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,0,1,0.0


In [19]:
# Criando uma variavel de Ano-Mes
df['data_ref'] = df['hora_ref'].apply(lambda x: x[:7])

## Tratando dados meteorologicos

Dados:  
- METAR (Meteorological Aerodrome Report) (Real)

- METAF (Terminal Aerodrome Forecast) (Previsto)

O METAR, sigla de Meteorological Aerodrome Report, é o informe meteorológico regular de aeródromo e contém as seguintes informações na sequência:

- Grupos de identificação;  
- Vento à superfície;  
- Visibilidade horizontal; 
- Alcance visual na pista (quando houver); 
- Tempo presente; 
- Nuvens (ou visibilidade vertical, se for o caso); 
- Temperaturas do ar e do ponto de orvalho; 
- Pressão atmosférica (QNH); e  
- Informações suplementares de inclusão condicional sobre tempo recente, cortante do vento, temperatura da superfície do mar, estado do mar e, por Acordo Regional de Navegação Aérea, o estado da pista.  

Grupo de Identificação 

METAR SBMN 061300Z 31015G27KT 280V350 5000 1500W -RA BKN010 SCT020 FEW025TCU 25/24 Q1014 RERA WS RWY17 W12/H75= 

METAR – Designador do código para informação meteorológica aeronáutica regular. Quando houver alguma mudança significativa fora do horário de confecção da mensagem METAR, o observador meteorológico deverá confeccionar a mensagem do tipo SPECI, sendo este campo substituído.

SBMN – código ICAO do aeroporto (S de América do Sul, B de Brasil e MN identificando o aeroporto de Manaus). 

061300Z – Dia do mês e horário da observação, em horas e minutos UTC, seguidos, sem espaço, da letra Z,  indicando o fuso horário ZULU. 



Alguns informes podem vir com a sigla COR apor o METAR, indicando uma correcao posterior daquele codigo, para esses casos os dados podem nao vir com a letra Z ao final do timestamp, portanto devemos tratar esse caso para decodificacao do informe.

In [47]:
# Preenchendo dados reais nulos por dados previstos
    # Alguns casos podem nao possuir tanto dados reais quanto previstos
df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), 'metaf']

# Substituir o texto METAF por METAR
df['metar'] = df['metar'].apply(lambda txt: re.sub('METAF', 'METAR', str(txt)))

In [48]:
# Funcao para ajustar dados meteorologicos com falta de informacao temporal
def metar_fix(txt):
    txt = str(txt)
    if txt != None and txt != 'nan':  
        first_position = txt.find('SB')
        second_position = txt[first_position+5:].find(' ')
        final_position = first_position+5+second_position
        if txt[final_position-1] != 'Z':
            txt = txt[:final_position] + 'Z' + txt[final_position:]
        return txt

df['metar'] = df['metar'].apply(metar_fix)

In [54]:
# Conversao do informe metar para uma tabela de dados
def convert_metar(df_met):
    df_final = pd.DataFrame()
    for i, row in df_met.iterrows():
        met, date = row['metar'], row['data_ref']
        try:
            aux = parse_metar_to_dataframe(met, year=int(date[:4]), month=int(date[5:7]))
            # aux.drop('station_id', axis=1, inplace=True)
            # aux.reset_index(inplace=True)
            aux.dropna(axis=0, how='all', inplace=True)
            if aux is not None:
                aux['meta'] = met
                aux['date'] = date
                # return aux
                df_final = pd.concat([df_final, aux])
        except: 
            print(met, date)
            break
    return df_final
    
    
df_metar = convert_metar(df[['metar', 'data_ref']].loc[~df['metar'].isna()].drop_duplicates())

C:\Users\pedro\AppData\Local\Temp\ipykernel_27956\3414867184.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, aux])
C:\Users\pedro\AppData\Local\Temp\ipykernel_27956\3414867184.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, aux])
C:\Users\pedro\AppData\Local\Temp\ipykernel_27956\3414867184.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this wil

In [56]:
df_metar

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,wind_gust,visibility,current_wx1,...,altimeter,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,remarks,air_pressure_at_sea_level,eastward_wind,northward_wind,meta,date
station_id,,,,,,,,,,,,,,,,,,,,,
SBFL,SBFL,-27.67,-48.53,5,2022-06-01 00:00:00,170.0,9.0,NaN,9999.0,NaN,...,30.12,0,0,0,,1020.30,-1.562834,8.863270,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,2022-06
SBCF,SBCF,-19.63,-43.97,828,2022-06-01 00:00:00,120.0,6.0,NaN,10000.0,NaN,...,30.06,0,0,0,,1015.39,-5.196152,3.000000,METAR SBCF 010000Z 12006KT CAVOK 21/14 Q1018=,2022-06
SBCT,SBCT,-25.52,-49.17,908,2022-06-01 00:00:00,100.0,6.0,NaN,7000.0,-RA,...,30.09,61,0,0,,1020.20,-5.908847,1.041889,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,2022-06
SBSV,SBSV,-12.90,-38.32,6,2022-06-01 00:00:00,110.0,8.0,NaN,9999.0,NaN,...,29.94,0,0,0,,1014.27,-7.517541,2.736161,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,2022-06
SBPA,SBPA,-30.00,-51.17,3,2022-06-01 00:00:00,NaN,3.0,NaN,10000.0,NaN,...,30.15,0,0,0,,1021.31,NaN,NaN,METAR SBPA 010000Z VRB03KT CAVOK 09/04 Q1021=,2022-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SBCF,SBCF,-19.63,-43.97,828,2023-05-31 21:00:00,70.0,7.0,NaN,9999.0,NaN,...,30.15,0,0,0,,1018.11,-6.577848,-2.394141,METAR SBCF 312100Z 07007KT 9999 BKN047 22/14 Q...,2023-05
SBSP,SBSP,-23.62,-46.63,803,2023-05-31 22:00:00,200.0,7.0,NaN,9999.0,NaN,...,30.21,0,0,0,,1022.23,2.394141,6.577848,METAR SBSP 312200Z 20007KT 170V230 9999 SCT009...,2023-05
SBCT,SBCT,-25.52,-49.17,908,2023-05-31 22:00:00,120.0,4.0,NaN,9999.0,NaN,...,30.21,0,0,0,,1022.34,-3.464102,2.000000,METAR SBCT 312200Z 12004KT 9999 SCT013 BKN050 ...,2023-05


In [60]:
df.merge(df_metar, how='left', left_on=['metar', 'data_ref'], right_on=['meta', 'date']).to_csv('metar.csv', index=False)

In [4]:
import pandas as pd
df = pd.read_csv('metar.csv')

C:\Users\pedro\AppData\Local\Temp\ipykernel_43876\2498234858.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('metar.csv')


In [5]:
metar = df[df.columns[11:-2]]
metar.isna().sum()

station_id                     1316
latitude                       1316
longitude                      1316
elevation                      1316
date_time                      1522
wind_direction                17539
wind_speed                     1320
wind_gust                    299979
visibility                     1334
current_wx1                  261360
current_wx2                  294028
current_wx3                  302362
low_cloud_type                82956
low_cloud_level               89348
medium_cloud_type            184053
medium_cloud_level           184053
high_cloud_type              266253
high_cloud_level             266253
highest_cloud_type           299824
highest_cloud_level          299824
cloud_coverage                 1316
air_temperature                1651
dew_point_temperature          1974
altimeter                      1626
current_wx1_symbol             1316
current_wx2_symbol             1316
current_wx3_symbol             1316
remarks                     

In [6]:
# Selecionando colunas que tenham no maximo 30% dos valores nulos para tratamento
valid_cols = metar.columns[(metar.isna().sum() <= metar.shape[0]*0.3).values]

metar_valid = metar[valid_cols]
metar_valid.dtypes

station_id                    object
latitude                     float64
longitude                    float64
elevation                    float64
date_time                     object
wind_direction               float64
wind_speed                   float64
visibility                   float64
low_cloud_type                object
low_cloud_level              float64
cloud_coverage               float64
air_temperature              float64
dew_point_temperature        float64
altimeter                    float64
current_wx1_symbol           float64
current_wx2_symbol           float64
current_wx3_symbol           float64
air_pressure_at_sea_level    float64
eastward_wind                float64
northward_wind               float64
dtype: object

In [12]:
metar_valid

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,visibility,low_cloud_type,low_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,air_pressure_at_sea_level,eastward_wind,northward_wind
0,SBFL,-27.67,-48.53,5.0,2022-06-01 00:00:00,170.0,9.0,9999.0,BKN,3000.0,6.0,14.0,7.0,30.12,0.0,0.0,0.0,1020.30,-1.562834,8.863270
1,SBFL,-27.67,-48.53,5.0,2022-06-01 00:00:00,170.0,9.0,9999.0,BKN,3000.0,6.0,14.0,7.0,30.12,0.0,0.0,0.0,1020.30,-1.562834,8.863270
2,SBCF,-19.63,-43.97,828.0,2022-06-01 00:00:00,120.0,6.0,10000.0,NaN,NaN,0.0,21.0,14.0,30.06,0.0,0.0,0.0,1015.39,-5.196152,3.000000
3,SBCT,-25.52,-49.17,908.0,2022-06-01 00:00:00,100.0,6.0,7000.0,BKN,400.0,8.0,10.0,10.0,30.09,61.0,0.0,0.0,1020.20,-5.908847,1.041889
4,SBSV,-12.90,-38.32,6.0,2022-06-01 00:00:00,110.0,8.0,9999.0,FEW,2300.0,2.0,27.0,21.0,29.94,0.0,0.0,0.0,1014.27,-7.517541,2.736161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302394,SBSP,-23.62,-46.63,803.0,2023-05-31 22:00:00,200.0,7.0,9999.0,SCT,900.0,8.0,16.0,14.0,30.21,0.0,0.0,0.0,1022.23,2.394141,6.577848
302395,SBKP,-23.00,-47.12,661.0,2023-05-31 22:00:00,200.0,4.0,9999.0,OVC,3500.0,8.0,18.0,16.0,30.18,0.0,0.0,0.0,1020.99,1.368081,3.758770
302396,SBKP,-23.00,-47.12,661.0,2023-05-31 22:00:00,200.0,4.0,9999.0,OVC,3500.0,8.0,18.0,16.0,30.18,0.0,0.0,0.0,1020.99,1.368081,3.758770
302397,SBRJ,-22.88,-43.17,3.0,2023-05-31 22:00:00,310.0,4.0,9999.0,FEW,700.0,6.0,20.0,19.0,30.03,0.0,0.0,0.0,1017.29,3.064178,-2.571150


Por definicao da biblioteca utilizada, cloud_coverage é uma conversão numérica para low_cloud_type
Guia de conversão https://en.wikipedia.org/wiki/Okta

In [16]:
metar_valid.drop('low_cloud_type', axis=1, inplace=True)

C:\Users\pedro\AppData\Local\Temp\ipykernel_888\1313770084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metar_valid.drop('low_cloud_type', axis=1, inplace=True)


In [17]:
metar_valid

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,visibility,low_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,air_pressure_at_sea_level,eastward_wind,northward_wind
0,SBFL,-27.67,-48.53,5.0,2022-06-01 00:00:00,170.0,9.0,9999.0,3000.0,6.0,14.0,7.0,30.12,0.0,0.0,0.0,1020.30,-1.562834,8.863270
1,SBFL,-27.67,-48.53,5.0,2022-06-01 00:00:00,170.0,9.0,9999.0,3000.0,6.0,14.0,7.0,30.12,0.0,0.0,0.0,1020.30,-1.562834,8.863270
2,SBCF,-19.63,-43.97,828.0,2022-06-01 00:00:00,120.0,6.0,10000.0,NaN,0.0,21.0,14.0,30.06,0.0,0.0,0.0,1015.39,-5.196152,3.000000
3,SBCT,-25.52,-49.17,908.0,2022-06-01 00:00:00,100.0,6.0,7000.0,400.0,8.0,10.0,10.0,30.09,61.0,0.0,0.0,1020.20,-5.908847,1.041889
4,SBSV,-12.90,-38.32,6.0,2022-06-01 00:00:00,110.0,8.0,9999.0,2300.0,2.0,27.0,21.0,29.94,0.0,0.0,0.0,1014.27,-7.517541,2.736161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302394,SBSP,-23.62,-46.63,803.0,2023-05-31 22:00:00,200.0,7.0,9999.0,900.0,8.0,16.0,14.0,30.21,0.0,0.0,0.0,1022.23,2.394141,6.577848
302395,SBKP,-23.00,-47.12,661.0,2023-05-31 22:00:00,200.0,4.0,9999.0,3500.0,8.0,18.0,16.0,30.18,0.0,0.0,0.0,1020.99,1.368081,3.758770
302396,SBKP,-23.00,-47.12,661.0,2023-05-31 22:00:00,200.0,4.0,9999.0,3500.0,8.0,18.0,16.0,30.18,0.0,0.0,0.0,1020.99,1.368081,3.758770
302397,SBRJ,-22.88,-43.17,3.0,2023-05-31 22:00:00,310.0,4.0,9999.0,700.0,6.0,20.0,19.0,30.03,0.0,0.0,0.0,1017.29,3.064178,-2.571150


In [ ]:
from datetime import timedelta

df['hora_ref'] = pd.to_datetime(df['hora_ref'])

def find_nearest_image(date):
    window = timedelta(hours=24)
    datas_proximas = (df['hora_ref']-date).abs()
    if (datas_proximas <= window).sum() > 0:
        nearest_row = df.iloc[datas_proximas.argsort()[0]]
        return nearest_row['url_img_satelite']
    
preenchimento = df[df['url_img_satelite'].isna()]['hora_ref'].apply(find_nearest_image)

preenchimento.unique()